<a href="https://colab.research.google.com/github/lhajabi/PPI_Grad/blob/main/GRU_PPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip3 install keras-visualizer
from keras_visualizer import visualizer

In [ ]:
"""ONLY RUNS IF GPU AVAILABLE
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

gpu()"""

# imports

In [ ]:
import tensorflow as tf
import keras
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__ if hasattr(tf.keras, '__version__') else tf.keras.__path__[0].split('/')[-1]}")
print(f"Standalone Keras version: {keras.__version__}")


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from itertools import product
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import json
from datetime import datetime

In [ ]:
from tensorflow import keras
from keras.layers import LSTM, GRU, Input, concatenate, Flatten, BatchNormalization, Dense, Dropout, Embedding, Reshape, RNN, SimpleRNN,Bidirectional
from keras.models import Model,Sequential
from keras.callbacks import Callback,EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score,roc_curve,auc,matthews_corrcoef,accuracy_score
from keras.metrics import Precision, Recall,Accuracy
from keras import layers

#Data Preperation:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_links = pd.read_csv('/content/drive/MyDrive/Data/GENERATED DATASETS/Generated100k_(POS40-0.7)_50:50.txt', delimiter=',')
df_links.info()

tokenization and padding:

In [ ]:
def generate_word_token_dict(amino_acids_list, k):
    words = set(product(amino_acids_list, repeat=k))
    words = [''.join(word) for word in words]
    tokens = [int(i) for i in range(1, len(words) + 1)]
    #num_digits = len(str(max(tokens)))
    #tokens = [f"{token:0{num_digits}}" for token in tokens]
    word_token = dict(zip(words, tokens))
    return word_token

def tokenization(cell, word_token, n_grams):
  if len(cell)/n_grams != 0:
    cell_0 = cell[len(cell)%n_grams:]
  cell_1 = [cell_0[i:i+n_grams] for i in range(0, len(cell_0), n_grams)]
  #print(cell_1)
  cell_1 = [word_token[word] for word in cell_1]
  #cell = ''.join(cell_1)
  cell = cell_1
  return cell

def pad(cell, fixed_length):
    if len(cell) > fixed_length:
        # If the original list is longer, remove leftmost elements
        return cell[-fixed_length:]
    elif len(cell) < fixed_length:
        # If the original list is shorter, pad leftmost with zeroes
        return [0] * (fixed_length - len(cell)) + cell
    else:
        # If the original list has the desired length, no modifications are needed
        return cell

In [ ]:
amino_acids_list = ['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y','Z']
n_grams = 3
fixed_length = 1000
word_token = generate_word_token_dict(amino_acids_list, k=n_grams)

In [ ]:

columns = ['protein1', 'protein2']
for column in columns:
  df_links[column] = df_links[column].apply(func = tokenization, args = (word_token, n_grams,))
  df_links[column] = df_links[column].apply(func= pad, args = (fixed_length,))
df_links.head()

In [ ]:
df_links.info()

#Split Data:

In [ ]:
x_data = df_links[['protein1', 'protein2']]
y_data = df_links['combined_score']
x_train, x_temp, y_train, y_temp = train_test_split(x_data, y_data, test_size=0.3, random_state=42, shuffle=True, stratify=None)
x_validate, x_test, y_validate, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=43, shuffle=True, stratify=None)

In [ ]:
print(x_train.info())
print(x_validate.info())
print(x_test.info())

In [ ]:
input_1_train = np.array(x_train['protein1'].values.tolist())
input_2_train = np.array(x_train['protein2'].values.tolist())
output_train = np.array(y_train)

input_1_validate = np.array(x_validate['protein1'].values.tolist())
input_2_validate = np.array(x_validate['protein2'].values.tolist())
output_validate = np.array(y_validate)

input_1_test = np.array(x_test['protein1'].values.tolist())
input_2_test = np.array(x_test['protein2'].values.tolist())
output_test = np.array(y_test)

In [ ]:
x_train = [input_1_train, input_2_train]
y_train = output_train

x_val = [input_1_validate, input_2_validate]
y_val = output_validate

x_test = [input_1_test, input_2_test]
y_test = output_test

In [ ]:
"""y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
y_test = y_test.astype('float32')"""

# Tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/

#Metrics and Callbacks:

In [ ]:
thresholds = [0.3, 0.5, 0.7, 0.9]
metrics = [ #Available metrics for binary_crossentropy are: loss,precision,recall,auc,auc_1,val_loss,val_precision,val_recall,val_auc,val_auc_1
    keras.metrics.BinaryAccuracy(threshold=0.5),
    keras.metrics.Accuracy(),
    keras.metrics.Precision(thresholds = thresholds),
    keras.metrics.Recall(thresholds = thresholds),
    keras.metrics.AUC(curve = 'ROC', name = 'ROC'),
    keras.metrics.AUC(curve="PR", name = 'PR'),
    #f1_score,
    keras.metrics.BinaryCrossentropy()
    ]
loss_function = 'binary_crossentropy'


In [ ]:
import keras.backend as K
"""def f1_score(y_true, y_pred):
    precision = Precision()(y_true, y_pred)
    recall = Recall()(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))
"""
def calculate_metrics(y_true, y_pred, threshold):
    y_pred_binary = (y_pred > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_binary).ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    mcc = matthews_corrcoef(y_true, y_pred_binary)
    accuracy = accuracy_score(y_true, y_pred_binary)
    precision = precision_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    auc = roc_auc_score(y_true, y_pred)

    return sensitivity, specificity, mcc, accuracy, precision, recall, f1, auc

In [ ]:
def find_best_threshold(model, x_val, y_val):
    y_pred = model.predict(x_val)

    # Generate a range of thresholds
    thresholds = np.arange(0, 1.01, 0.01)
    best_threshold = None
    best_mcc = -1

    # Iterate through thresholds and find the best one
    for threshold in thresholds:
        sens,spec,mcc,acc,prec, rec, f1,auc = calculate_metrics(y_val, y_pred, threshold)
        if mcc > best_mcc:
            best_mcc = mcc
            best_threshold = threshold

    return best_threshold,best_mcc

In [ ]:
class ConfusionMatrixCallback(keras.callbacks.Callback):
  def __init__(self, x_val, y_val):
        super(ConfusionMatrixCallback, self).__init__()
        self.x_val, self.y_val = x_val, y_val
  def on_epoch_end(self, epoch, logs=None):
        predictions = self.model.predict(x = x_val)
        # Convert probabilities to classes using the threshold
        predictions = (predictions >= 0.5).astype(int)
        cm = confusion_matrix(y_val, predictions)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
        plt.title('Confusion Matrix - Epoch {}'.format(epoch))
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()

In [ ]:
def rate_model(y_true, y_pred, cm_threshold = 0.5):
  for m in metrics:
    m.update_state(y_true, y_pred)
    print('{} : {}'.format(m.name, m.result()))
  y_pred = (y_pred >= cm_threshold).astype(int)
  cm = confusion_matrix(y_true, y_pred)
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
  plt.title('Confusion Matrix')
  plt.xlabel('Predicted')
  plt.ylabel('True')
  plt.show()

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor= 'val_loss' , factor=0.2, patience=0.1, min_lr=0.001, verbose=1)
confusion_matrix_callback = ConfusionMatrixCallback(x_val, y_val)
early_stopping = EarlyStopping(monitor= 'val_loss', patience=5, restore_best_weights = True)
callbacks = [early_stopping, reduce_lr, confusion_matrix_callback]

#GRU_1:

In [ ]:
#inputs
protein_1 = tf.keras.layers.Input(shape=(fixed_length,), name='protein1')
protein_2 = tf.keras.layers.Input(shape=(fixed_length,), name='protein2')

In [ ]:
#shared embedding layer
embedding = tf.keras.layers.Embedding(input_dim = len(word_token) + 1, output_dim = 512, mask_zero = 1,  input_length=fixed_length, name = 'shared_embedding_0')
embedding_1 = embedding(protein_1)
embedding_2 = embedding(protein_2)
#concatenated_embeddings = concatenate([embedding_1, embedding_2], name = 'shared_embedding_1')
model_0 = Model(inputs = [protein_1, protein_2], outputs = [embedding_1,embedding_2])

model_1 = tf.keras.Sequential(name = 'sequential')
model_1.add(tf.keras.layers.BatchNormalization(name = 'bn_0', input_shape = (fixed_length,512)))
model_1.add(tf.keras.layers.GRU(name = 'gru', units = 64, activation='tanh', recurrent_activation='sigmoid', go_backwards = True , return_sequences= False ))
model_1.add(tf.keras.layers.Dense( 64, activation='elu',name = 'dense'))
model_1.add(tf.keras.layers.BatchNormalization(name = 'bn_1'))
model_1.add(tf.keras.layers.Dropout(0, name = 'dropout'))

feature_array_128 = tf.keras.layers.concatenate([model_1(model_0.outputs[0]), model_1(model_0.outputs[1])])
x = tf.keras.layers.Dense(512, activation='elu')(feature_array_128)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x) #i added this because we changed the loss function
final_model = Model(inputs = [protein_1, protein_2], outputs = x)
final_model.summary()

#GRU:

In [ ]:
#inputs
protein_1 = tf.keras.layers.Input(shape=(fixed_length,), name='protein1')
protein_2 = tf.keras.layers.Input(shape=(fixed_length,), name='protein2')

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim = len(word_token) + 1, output_dim = 512, mask_zero = 1,  input_length=fixed_length,name='Embedding')
embedding_1 = embedding(protein_1)
embedding_2 = embedding(protein_2)
model_0 = Model(inputs = [protein_1, protein_2], outputs = [embedding_1,embedding_2])

model_1 = Sequential()
model_1.add(BatchNormalization(name = 'BatchNormalization0' ,input_shape = (fixed_length,512)))
model_1.add(GRU(name = 'GRU_Layer',units = 64, activation='tanh', recurrent_activation='sigmoid'))
model_1.add(Dense( 64, activation='elu',name = 'Dense'))
model_1.add(BatchNormalization( name = 'BatchNormalizatio1n' ))
model_1.add(Dropout(0.4))

feature_array_128 = concatenate([model_1(model_0.outputs[0]), model_1(model_0.outputs[1])])
x = Dense(512, activation='elu',name = 'Dense0')(feature_array_128)
x = BatchNormalization(name = 'BatchNormalization' )(x)
x = Dense(1, activation='sigmoid',name = 'Dense1')(x)
final_model = Model(inputs = [protein_1, protein_2], outputs = x)
final_model.summary()


In [ ]:
 #visualizer(final_model, file_format='png', view=True)

In [ ]:
"""from tensorflow.keras.utils import plot_model
plot_model(final_model, to_file='model.png', show_shapes=True, show_layer_names=True)
"""

"from tensorflow.keras.utils import plot_model\nplot_model(final_model, to_file='model.png', show_shapes=True, show_layer_names=True)\n"

In [ ]:
final_model.compile(optimizer='adam',loss=loss_function, metrics = metrics)

In [ ]:
final_model.load_weights('/content/drive/MyDrive/saved_models/2024_01_06_04_18/final_weights_GRU3.h5')

#Training:

In [ ]:
final_model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 3, callbacks = callbacks,batch_size=100)

In [ ]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
drive_path = os.path.join('/content/drive/MyDrive/saved_models', timestamp)
!mkdir -p $drive_path

In [ ]:
final_model.save_weights(os.path.join(drive_path, f'final_weights_GRU3.h5'))
final_model.save(os.path.join(drive_path, f'final_model_GRU3.h5'))

#TensorBoard:

In [ ]:
%tensorboard --logdir logs/fit

#GRU Bidirectional:

In [ ]:
#inputs
protein_1 = tf.keras.layers.Input(shape=(fixed_length,), name='protein1')
protein_2 = tf.keras.layers.Input(shape=(fixed_length,), name='protein2')

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim = len(word_token) + 1, output_dim = 512, mask_zero = 1,  input_length=fixed_length)
embedding_1 = embedding(protein_1)
embedding_2 = embedding(protein_2)
model_0 = Model(inputs = [protein_1, protein_2], outputs = [embedding_1,embedding_2])

model_1 = Sequential()
model_1.add(BatchNormalization(input_shape = (fixed_length,512)))
model_1.add(Bidirectional(GRU(name = 'GRU_Layer',units = 64, activation='tanh', recurrent_activation='sigmoid')))
model_1.add(Dense( 64, activation='elu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.6))

feature_array_128 = concatenate([model_1(model_0.outputs[0]), model_1(model_0.outputs[1])])
x = Dense(512, activation='elu')(feature_array_128)
x = BatchNormalization()(x)
x = Dense(1, activation='sigmoid')(x)
final_model = Model(inputs = [protein_1, protein_2], outputs = x)
final_model.summary()


In [ ]:
final_model.compile(optimizer='adam',loss=loss_function, metrics = metrics)

In [ ]:
final_model.load_weights('/content/drive/MyDrive/saved_models/2024_01_09_03_05/final_weights_GRUB3.h5')

In [ ]:
final_model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 3, callbacks = callbacks,batch_size=64)

In [ ]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
drive_path = os.path.join('/content/drive/MyDrive/saved_models', timestamp)
!mkdir -p $drive_path

In [ ]:
final_model.save_weights(os.path.join(drive_path, f'final_weights_GRUB3.h5'))
final_model.save(os.path.join(drive_path, f'final_model_GRUB3.h5'))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Rate model performance:

In [ ]:
#Find the best threshold based on best mcc score on val data
best_threshold,best_MCC = find_best_threshold(final_model, x_val, y_val)
print(f'Best Matthews Correlation Coefficient: {best_MCC} at Threshold: {best_threshold}')


In [ ]:
def evaluate_with_threshold(model, x_test, y_test, threshold):
    y_pred = model.predict(x_test)
    sens,spec,mcc,acc,prec, rec, f1,auc = calculate_metrics(y_test, y_pred, threshold)
    print(f'For Threshold:{threshold}:\nMCC = {mcc}\nsensitivity = {sens}\nspecificity = {spec}\naccuracy = {acc}\nprecision = {prec}\nrecall = {rec}\nf1-score = {f1}\nauc = {auc}')

# Evaluate the model with the best threshold
evaluate_with_threshold(final_model, x_test, y_test, best_threshold)


In [ ]:
predictions = final_model.predict(x = x_test, callbacks = None )

469/469 [==============================] - 210s 448ms/step


In [ ]:
rate_model(y_test, predictions)

In [ ]:
"""
# Load the evaluation results
evaluation_results_path = os.path.join(drive_path, 'evaluation_results.json')
with open(evaluation_results_path, 'r') as file:
    evaluation_results = json.load(file)
"""
y_true = output_test
y_pred = predictions

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred)

# Compute area under the curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


#Load model & weights:

In [ ]:
#run to load pretrained weights
new_model = final_model
new_model.load_weights('/content/drive/MyDrive/saved_models/2024_01_02_20_28/final_weights_GRU5.h5')
#new_model.summary()
new_model.compile(optimizer='adam',loss=loss_function, metrics = metrics)

In [ ]:
new_model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 3, callbacks = callbacks,batch_size=100)

Epoch 1/3


In [ ]:
"""
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
drive_path = os.path.join('/content/drive/MyDrive/saved_models', timestamp)
!mkdir -p $drive_path

new_model.save_weights(os.path.join(drive_path, f'final_weights_GRU6.h5'))
new_model.save(drive_path,'final_model_GRU6.h5')"""

In [ ]:
evaluate_train_0 = new_model.evaluate(x = x_test_temp, y = y_test_temp, callbacks = None )

In [ ]:
predictions_0 = new_model.predict(x = x_test, callbacks = None )

In [ ]:
def rate_model(y_true, y_pred, cm_threshold = 0.5):
  for m in metrics:
    m.update_state(y_true, y_pred)
    print('{} : {}'.format(m.name, m.result()))
  y_pred = (y_pred >= cm_threshold).astype(int)
  cm = confusion_matrix(y_true, y_pred)
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=True, yticklabels=True)
  plt.title('Confusion Matrix')
  plt.xlabel('Predicted')
  plt.ylabel('True')
  plt.show()

rate_model(y_test, predictions_0)

#Tester:

In [ ]:
import joblib

joblib.dump(final_model, 'model_GRU_final.joblib', protocol=5)

['model_GRU_final.joblib']

In [ ]:
import cloudpickle

with open('model_GRU_final.pkl', 'wb') as file:
    cloudpickle.dump(final_model, file)